In [1]:
## IMPOR RELEVANT LIBRAIES

# DATA WRANGLING AND ANALYSIS
import pandas as pd
import missingno
from   skimpy import skim
import datetime as dt
import numpy as np
from random import choice 

# DATA VISULIZATION 
import seaborn  as sns
import matplotlib.pyplot as plt

# DATE MANIPULATION
#from dateutil.relativedelta import relativedelta

In [2]:
# import relevant datasets 

# CUSTOMER DEMOGRAPHIC DATASET
cusDemo    = pd.read_excel('KPMG_VI_New_raw_data_update_final.xlsx', sheet_name = 3 , parse_dates=True)

#Transation DATA
transc     = pd.read_excel('KPMG_VI_New_raw_data_update_final.xlsx', sheet_name = 1, parse_dates=True) 

#CUSTOMER ADDRESS
cusAddress = pd.read_excel('KPMG_VI_New_raw_data_update_final.xlsx', sheet_name = 4, parse_dates=True) 

C:\Users\Owner\AppData\Local\Temp\ipykernel_9908\2218133990.py:4: FutureWarning: Inferring datetime64[ns] from data containing strings is deprecated and will be removed in a future version. To retain the old behavior explicitly pass Series(data, dtype=datetime64[ns])
  cusDemo    = pd.read_excel('KPMG_VI_New_raw_data_update_final.xlsx', sheet_name = 3 , parse_dates=True)


#### DATA CLEANING - CUSTOMER DEMOGRAPHICS

In [4]:
cusDemo_cleaned              = cusDemo.copy() 
cusDemo_cleaned.drop(columns = ['first_name', 'last_name', 'default'], inplace = True)
y  = 2017    
cusDemo_cleaned.DOB.fillna(cusDemo_cleaned.DOB.mode()[0], inplace = True)
cusDemo_cleaned.DOB = round(y - cusDemo_cleaned.DOB.dt.year).astype(int)
cusDemo_cleaned     = cusDemo_cleaned[cusDemo_cleaned.DOB < 100]

#GROUP AGE(Countinous) into discrete variables 
bins    = np.arange(7.5,97.5,5)
labels_ = np.arange(10,95,5)
cusDemo_cleaned['AgeClass'] = pd.cut(cusDemo_cleaned.DOB, bins, labels = labels_)
#cusDemo_cleaned[['AgeClass', 'DOB']]

# Function to fill missing values with random integers between 1 and 22
def fill_missing_with_random_row(row):
    return row.apply(lambda x: np.random.randint(1, 22) if pd.isnull(x) else x)

# Apply the function to the specified row
cusDemo_cleaned.tenure = fill_missing_with_random_row(cusDemo_cleaned.tenure)


"""Convert thsis statement direclty to a pandas dataframe instead of converting to csv first"""
cusDemo_cleaned[["wealth_segment","job_title","job_industry_category"]].groupby(["wealth_segment","job_industry_category"]).value_counts().to_csv("check_dist.csv")
dictw = pd.read_csv("check_dist.csv")

dictw[(dictw.wealth_segment == "Affluent Customer") 
      & (dictw.job_title == "Editor")].job_industry_category.to_list()

# Function to fill missing values with random integers between 1 and 22
def fill_missing_with_random_row(row):
    return row.apply(lambda x: np.random.randint(1, 22) if pd.isnull(x) else x)


# Function to fill missing values in column C
dictw   = pd.read_csv("check_dist.csv")
jic_all = list(dictw.job_industry_category.unique())
cusDemo_cleaned 

def fill_missing_values(cusDemo_cleaned):
    for index, row in cusDemo_cleaned.iterrows():
        if pd.isnull(row['job_industry_category']):
            jic = dictw[(dictw.wealth_segment == row['wealth_segment']) & (dictw.job_title == row['job_title'])].job_industry_category.to_list()
            if len(jic) < 1:
                 cusDemo_cleaned.at[index, 'job_industry_category'] = choice(jic_all)
            else :
                cusDemo_cleaned.at[index, 'job_industry_category'] = choice(jic)
    return cusDemo_cleaned

# Apply the function to the DataFrame
cusDemo_cleaned = fill_missing_values(cusDemo_cleaned)

In [47]:
#Yet to be tested
cusDemo_cleaned.job_title.fillna("Classified")

#yet to find disparities in gender #U = unspecified gender 
#

In [ ]:
cusDemo_cleaned = cusDemo.copy() 

##### DROP IRRELEVANT COLUMNS
'''
Drop  default column, i doesnt have a meaning
Drop  default first and last names, Name will not influence purchase pattern
'''
cusDemo_cleaned.drop(columns = ['first_name', 'last_name', 'default'], inplace = True)
#### CLEAN DATE OF BIRTH COLUMN
""" 
- Convert Date of birth to Age 
- Drop null values
- Find and drop outliers
"""

# CONVERT DATE OF BIRTH TO AGE AND CLASSIFY IN TENS 
#take the maximum transcatodedate as the time of the project 
y  = 2017    
"""Fill missing age with mode"""
cusDemo_cleaned.DOB.fillna(cusDemo_cleaned.DOB.mode()[0], inplace = True)
cusDemo_cleaned.DOB = round(y - cusDemo_cleaned.DOB.dt.year).astype(int)
#It seen there is an outlier Age, Lets find out if there are more than one customer that is more than  100 years old 

"""
Oops!, her just caught an oulier.
In a real life situation, we can verify the customer age bu in this case, i will drop the customer from our database 
and the null date of births
"""
cusDemo_cleaned = cusDemo_cleaned[cusDemo_cleaned.DOB < 100]
"""Classify age into bins"""

# Clean job_title','job_industry_category Columns
""""
- Find cell is missing values
- Drop nulls in both category ---> 'job_title','job_industry_category'
"""

cusDemo_cleaned.job_industry_category.isna().value_counts()
cusDemo_cleaned.dropna(how = 'any')

# Attain unique job title to job industry 
job_to_title = cusDemo_cleaned[['job_title','job_industry_category']].dropna().drop_duplicates().reset_index(drop = True)

job_to_title[job_to_title.job_title == 'Blank']
              
test = cusDemo[['customer_id','DOB']].dropna()
test['AGE'] = 2017 - test.DOB.dt.year
test.AGE.astype(int)

test = test[test != test.AGE.max()]

bins = np.arange(10,100,10)
labels_ = np.arange(10,90,10)
test['Grouping'] = pd.cut(test.AGE, bins, labels = labels_ )
test


#### DATA CLEANING - address data 

In [7]:
cusAddress_cleaned = cusAddress.drop(["country"],axis =1) 
cusAddress_cleaned.state[cusAddress_cleaned.state == "New South Wales"] = "NSW"
cusAddress_cleaned.state[cusAddress_cleaned.state == "Victoria"] = "VIC"

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3999 entries, 0 to 3998
Data columns (total 5 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   customer_id         3999 non-null   int64 
 1   address             3999 non-null   object
 2   postcode            3999 non-null   int64 
 3   state               3999 non-null   object
 4   property_valuation  3999 non-null   int64 
dtypes: int64(3), object(2)
memory usage: 156.3+ KB


#### DATA MERGING  - address and demographics data 

In [14]:
customer_data = cusDemo_cleaned.merge(cusAddress_cleaned, how = "inner", on = "customer_id")

,customer_id,first_name,last_name,gender,past_3_years_bike_related_purchases,DOB,job_title,job_industry_category,wealth_segment,deceased_indicator,default,owns_car,tenure,address,postcode,state,property_valuation
0,1,Laraine,Medendorp,F,93,1953-10-12,Executive Secretary,Health,Mass Customer,N,"""'",Yes,11.0,060 Morning Avenue,2016,New South Wales,10
1,2,Eli,Bockman,Male,81,1980-12-16,Administrative Officer,Financial Services,Mass Customer,N,<script>alert('hi')</script>,Yes,16.0,6 Meadow Vale Court,2153,New South Wales,10
2,4,Talbot,NaN,Male,33,1961-10-03,NaN,IT,Mass Customer,N,() { _; } >_[$($())] { touch /tmp/blns.shellsh...,No,7.0,0 Holy Cross Court,4211,QLD,9
3,5,Sheila-kathryn,Calton,Female,56,1977-05-13,Senior Editor,NaN,Affluent Customer,N,NIL,Yes,8.0,17979 Del Mar Point,2448,New South Wales,4
4,6,Curr,Duckhouse,Male,35,1966-09-16,NaN,Retail,High Net Worth,N,ðµ ð ð ð,Yes,13.0,9 Oakridge Court,3216,VIC,9
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3991,3996,Rosalia,Halgarth,Female,8,1975-08-09,VP Product Management,Health,Mass Customer,N,-100,No,19.0,0 Transport Center,3977,VIC,6
3992,3997,Blanch,Nisuis,Female,87,2001-07-13,Statistician II,Manufacturing,High Net Worth,N,â¦testâ§,Yes,1.0,4 Dovetail Crossing,2350,NSW,2
3993,3998,Sarene,Woolley,U,60,NaT,Assistant Manager,IT,High Net Worth,N,NaN,No,NaN,736 Roxbury Junction,2540,NSW,6
3994,3999,Patrizius,NaN,Male,11,1973-10-24,NaN,Manufacturing,Affluent Customer,N,Â¡â¢Â£Â¢âÂ§Â¶â¢ÂªÂºââ,Yes,10.0,1482 Hauk Trail,3064,VIC,3


#### DATA CLEANING - transcation data 

In [18]:
# Copy the transaaction data 
trans_cleaned = transc.copy()

#Did customerss buy similar items??  - different items were purchased at diffrent timers 
len(trans_cleaned.product_id.unique())

101

In [47]:
# Investigate if there multiple transactions recorded 
trans_cleaned.transaction_id.duplicated().value_counts()


# Convert product_first_sold_date to the right format 
trans_cleaned['product_first_sold_date'] = pd.to_datetime(trans_cleaned['product_first_sold_date'], 
                                           unit = 'D',
                                           origin ='1899-12-30' )


#Convert online type to Boolean
def changeBoolean(var):
    if var == 0.0:
        return False
    elif var == 1.0:
        return True
    
trans_cleaned['online_order'] = trans_cleaned['online_order'].apply(changeBoolean)

# ADD a profit column and profit ratio
trans_cleaned['profit']      = trans_cleaned['list_price'] - trans_cleaned['standard_cost']
trans_cleaned['profitRatio'] = trans_cleaned['list_price'] / trans_cleaned['standard_cost']

# drop columns with null profit 
trans_cleaned          = trans_cleaned[trans_cleaned['profit'].notna()]
trans_cleaned.info()


## ADD AN UNSPECIFIED COLUMN TO online_order that is na
trans_cleaned.online_order.fillna('Unspecified', inplace=True)

# Find all categorical columns 
columns = [i for i in list(trans_cleaned.columns) if trans_cleaned[i].dtype == 'O']

for item in columns:
    print(f'{item} unique Values {trans_cleaned[item].unique()}')
    

# Observe thatcertain transactions were cancelled and note that Cancelled transactions dont add up to the revenue
trans_cleaned.order_status.value_counts() 

# Use only approved transcations 
trans_cleaned = trans_cleaned[trans_cleaned.order_status == 'Approved']


False    20001
True         1
Name: transaction_id, dtype: int64

#### THE  RELEVANT METRICS I USE WILL BE 
- TOTAL AND AVERAGE REVENUE PER CUSTOMER 
- FREQUENCY OF TRANSCATION 
- AVERAGE PROFIT RATION  PER TRANSACTION 

In [53]:
transcMetrics = trans_cleaned.groupby('customer_id').agg({'profit' : ['sum', 'mean'],
                                          "customer_id" : 'count' ,
                                          'profitRatio' : ['sum', 'mean']})



#### DATA CLEANING
ALL DATA SETS ARE CLEANED INORDER TO BE PREPROCESSED.
REDUNDANT COLUMNS ARE DROPPED, MISSING VALUES ARE FILLED, CATEGORIES ARE CORRECTED. 

#### DATA WRANGLING
DATA IS WRANGLED TO FIT INTO ONE ONE DATASET 

#### COMPUTATION
- COMPUTE AGE AND CLASSIFY BY AGE BRACKET

- AN APPROPRIATE METHOD IS USED TO CLASSIFY CURRENT TARGET- 1,0 WHICH WILL CORRESEPOND TO TRUE OR FALSE
  PROBLEM?? -- EHAT METHOD CAN I USE TO DESIGN A MODEL THAT CAN EFFECTIVELY CATEGORISE TH

#### VISUALIZATION AND CLASSIFICATION OF PUCAHSE BEHAVIOURS 
EDA IS CARRIED OUT O THE EXISTING DATASET AND MORE UNDERSTANDING OF THE PURCHASE BEHAVIOURS IS ATTAINED 

#### FEATURE ENGINEERING 
THE DATA SET IS ACCUSTOMED TO BE FIT INTO A MODEL

#### DEVELOP MODEL 
IN THIS CASE, WE WILL USE A CLASSICAL CLASSIFICTION MODEL TO CLASSIFY CUSKTOMERS INTO TWO GROUPS BASED ON THEIR PURCHASE BEHAVIOUS 

#### MODEL TESTING AND VALIDATION 
THE MODEL IS TESTED WITH THE TEST DATA TO VALIDATE THE CORRECTIES NESSA ND ACCUERACY

#### MODEL DEPLOYMENT
APPLY MODEL ON THE NEW CUSTOMERS FOR CLASSIFICATION